# Time to Peak

In [1]:
\c 20 70
\l ml/ml.q
.ml.loadfile`:init.q

In [2]:
/convert site numbers to 8 digits or longer
prepsite:{`${$[7=count x;"0",;]x}each string x}

In [3]:
/functions to read in nlcd data
nlcdtyp:("SFFFFFSFFSFSFFFFSFSSFSSFFFSFFSFF";enlist",")
nlcupd:{[y;t]
 col:`site_no`INTPTLAT`INTPTLON`Measure`REACHCODE`distance`imp`year;
 impcol:`$"imp_nlcd_",string 2000+y;
 ?[t;();0b;col!((prepsite;`SOURCE_FEA);`INTPTLAT;`INTPTLON;`Measure;`REACHCODE;`distance;impcol;y)]}
nlcdread:{nlcupd[x]nlcdtyp 0:hsym`$"data/snap_sampled_imp_nlcd_",string[2000+x],".csv"}

In [4]:
/nlcd data for 2006, 2011 and 2016
nlcd:raze nlcdread each 6 11 16

In [5]:
/remaining datasets
basin:("S",242#"F";enlist",")0:`:data/gages_with_basin_attr.csv
maxht:("SDF";enlist",")0:`:data/max_str_ht.csv
sites:exec prepsite site_no from(27#"S";enlist",")0:`:data/usgs_gage_subset.csv

In [6]:
\l t_p

In [7]:
show peak:select from peak where date>=2009.07.01,site_no in sites

date       site_no  lat      lon       start_time              end_..
-------------------------------------------------------------------..
2009.07.02 01377500 40.99278 -74.02111 2009.07.02T15:45:00.000 2009..
2009.07.02 01397000 40.57222 -74.86806 2009.07.02T11:00:00.000 2009..
2009.07.02 01396500 40.67778 -74.87917 2009.07.02T08:30:00.000 2009..
2009.07.07 06478500 43.18583 -97.63528 2009.07.07T12:45:00.000 2009..
2009.07.10 06478690 43.41556 -97.20472 2009.07.10T09:00:00.000 2009..
2009.07.11 04215000 42.89    -78.64528 2009.07.11T21:45:00.000 2009..
2009.07.11 04215500 42.82972 -78.77528 2009.07.11T21:15:00.000 2009..
2009.07.15 06438500 44.52944 -101.9297 2009.07.15T01:00:00.000 2009..
2009.07.18 06834000 40.35167 -101.1236 2009.07.18T04:00:00.000 2009..
2009.07.21 01391000 40.99778 -74.11194 2009.07.21T18:45:00.000 2009..
2009.07.23 02162093 34.02722 -81.04194 2009.07.23T04:00:00.000 2009..
2009.07.29 01391000 40.99778 -74.11194 2009.07.29T00:30:00.000 2009..
2009.07.29 01394500 

In [8]:
/read in precipall data
ppcols:`site_no`long`lat`elv`date`ppt
ppfiles:hsym`$"../data/prism/",/:string key`:../data/prism
precipall:`date xasc ppcols xcol -1_raze{10_("SFFFDF";enlist",")0:x}each ppfiles
precipall:update prepsite site_no from delete from precipall where site_no=`

In [9]:
/driverless feature lag function
featurelag:{[x;c;l]
 if[(::)~c;c:i.fndcols[x;"f"]];
 v:raze{y xprev'x}[x c,:()]each l,:();
 sl:{$[x<0;"neg_",1_;]string x}each l;  / add to OG function
 max[l]_flip flip[x],(raze`$string[c],\:/:"_lag_",/:sl)!v}

In [10]:
/add lag columns (past and future)
psite:exec distinct site_no from precipall
pptlag:{[t;l;s]featurelag[;`ppt;l]select from t where site_no=s}
precipall_fut:raze pptlag[precipall;(til[21]-10)except 0]each psite
precipall_fut:update avgppt:60 mavg ppt by site_no from precipall_fut

In [11]:
/join tables
peak_basin:(peak lj`site_no`date xkey precipall_fut)ij`site_no xkey basin

In [31]:
/add year column and merge
stryear:{$[x<2011;6;x<2016;11;16]}
merged:update year:stryear each`year$date from peak_basin
peak_bas_imp:merged ij`site_no`year xkey nlcd

In [32]:
/time tab, with sin and cos times and co-ords
pi:acos -1
time_peak:update month:`mm$date from peak_bas_imp
time_peak:update sin_t:sin 2*pi*month%11,cos_t:cos 2*pi*month%11,
 x:cos[lat]*cos lon,y:cos[lat]*sin lon,z:sin lat from time_peak
time_peak:update delta_peak:24*peak_time-start_time from time_peak

In [33]:
/lag height columns
htlag:{[t;i]d:i 0;c:i 1;featurelag[;`height;1 2]select from t where date=d,catch=c}
catchht:update catch:`$2#'string site_no from maxht
htinput:(cross/)(distinct catchht`date;distinct catchht`catch)
upstr:raze htlag[catchht]each htinput

In [34]:
time_peak:time_peak ij`site_no`date xkey upstr

In [35]:
opp:`ppt`ppt_lag_1`ppt_lag_2`ppt_lag_3`ppt_lag_4`ppt_lag_5`ppt_lag_6`ppt_lag_7`ppt_lag_8`ppt_lag_9`ppt_lag_10
npp:`prev1`prev2`prev3`prev4`prev5`prev6`prev7`prev8`prev9`prev10
oht:`height`height_lag_1`height_lag_2

In [36]:
maxlag:{[t;oc;nc]t,'flip nc!{max each?[x;();0b;y!y]}[t]each 1_{x,y}\[oc]}

In [39]:
/find max of each set of lag cols
time_peak:maxlag[time_peak;opp;npp]
upstr:maxlag[upstr;oht;`upst1`upst2]

In [40]:
/mark training and testing sets
\S 42
sitesplit:`trn`tst!s(0,floor n*1-.2)_neg[n]?n:count s:distinct time_peak`site_no
time_peak:update split:{$[x in sitesplit`trn;`trn;`tst]}each site_no from time_peak

In [41]:
/define classes
time_peak:update target:{$[x<3.12;0;x<7.44;1;x<18;2;3]}each delta_peak from time_peak

In [42]:
\l ../gagesdir/gagesdir

In [43]:
maxmovingavg:{[x;c;l]
 v:max each raze{y mavg'x}[x c,:()]each l,:();
 (raze`$"wind_",/:string l)!v}

In [44]:
/find max moving avg of wind data
sdt:value each select site_no,start_time from time_peak
strtabs:{sn:x 0;st:x 1;r:(st;st+1);select from str where date within"d"$r,datetime within r,sn=`$site_no}each sdt
wind_ht_24:maxmovingavg[;`height;2 4 12 48]each strtabs

In [45]:
time_peak_ht:time_peak,'wind_ht_24

## Multi-Class

In [46]:
mattab:{flip value flip x}

In [47]:
plt:.p.import`matplotlib.pyplot
fetch:.p.import[`sklearn.datasets]`:fetch_mldata
rfc:.p.import[`sklearn.ensemble]`:RandomForestClassifier
ovr:.p.import[`sklearn.multiclass]`:OneVsRestClassifier
pr_curve:.p.import[`sklearn.metrics]`:precision_recall_curve
avg_precision:.p.import[`sklearn.metrics]`:average_precision_score

/home/deanna/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [50]:
cols_to_include:
 `imp`CatAreaSqKm`WsAreaSqKm`CatAreaSqKmRp100`WsAreaSqKmRp100`ElevCat`ElevWs,
 `WtDepCat`WtDepWs`OmCat`OmWs`PermCat`PermWs`RckDepCat`RckDepWs`ClayCat`ClayWs,
 `SandCat`SandWs`RunoffCat`RunoffWs`WetIndexCat`WetIndexWs`BFICat`BFIWs,
 `ppt`avgppt`ppt_lag_neg_1`ppt_lag_neg_2`upst1`upst2,
 `sin_t`cos_t`x`y`z,
 `prev1`prev2`prev3`prev4`prev5`prev6`prev7`prev8`prev9`prev10,
 `next1`next2`next3`next4`next5`next6`next7`next8`next9`next10,
 `wind_2`wind_4`wind_12`wind_48

In [63]:
time_peak_clean:time_peak_ht where not any each null time_peak_ht

In [65]:
time_peak_clean

date       site_no  lat     lon       start_time              end_t..
-------------------------------------------------------------------..
2009.11.03 02037500 37.5632 -77.54722 2009.11.03T00:30:00.000 2009...
2009.11.12 02037500 37.5632 -77.54722 2009.11.12T20:00:00.000 2009...
2009.11.12 02055000 37.2585 -79.93889 2009.11.12T10:15:00.000 2009...
2009.11.12 02075045 36.5626 -79.37    2009.11.12T01:00:00.000 2009...
2009.11.12 02060500 37.1046 -79.29556 2009.11.12T11:30:00.000 2009...
2009.11.13 02047000 36.7704 -77.16639 2009.11.13T23:45:00.000 2009...
2009.11.20 02037500 37.5632 -77.54722 2009.11.20T16:00:00.000 2009...
2009.12.04 02037500 37.5632 -77.54722 2009.12.04T03:30:00.000 2009...
2009.12.06 02047000 36.7704 -77.16639 2009.12.06T02:15:00.000 2009...
2009.12.09 02037500 37.5632 -77.54722 2009.12.09T13:15:00.000 2009...
2009.12.09 01667500 38.3504 -77.97528 2009.12.09T17:30:00.000 2009...
2009.12.09 01664000 38.5307 -77.81389 2009.12.09T16:15:00.000 2009...
2009.12.09 02055000 

In [55]:
time_peak_clean:delete from time_peak_ht where i in where any flip null value each time_peak_ht

1100111100000011000001100000000000000000000000000000000000000000000..
